In [5]:
import os
os.chdir('/home/roberto/Documents/Titulación/Tesina')
from scripts import *

### Países disponibles:

##### -España
##### -Alemania

### Años disponibles:

##### 2008 a 2017
### Métodos disponibles

solar_one_day(country,year,month,day)

correr_simulacion() para ejecutar simulación de nuevo

leer() leer como DataFrame la salida de la simulación

area_elegible(country, includer) para mapa cuadriculado del área a utilizar para paneles (Includer disponibles = dump y water)

tres_random(pais) para gráfica de tres días aleatorios de generación de energía

anual_country(pais) Generación anual por país. Regresa DF

suma_mes(country,mes) Regresa la producción total de un mes para cada año en un país (Regresa diccionario)

graficas_bienestar(country) Diagrama de caja y brazos para un país

produccion_por_mes(country) Producción por mes. Regresa la media y su estimación con intervalos de confianza al 95% (DataFrame)

ventas_spain() Regresa DataFrame con las ventas por cada mes de España

ventas_germany() Regresa DataFrame con las ventas por cada mes de Alemania

graficas_cuatrimestre(country) Gráficas por cuatrimestre. Perc.0.75, median y perc. 0.25